In [1]:
import numpy as np
from tifffile import imread, imwrite 
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile
import scipy.ndimage as ndi
import csv
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from skimage.filters import threshold_otsu
import cv2
from skimage.measure import regionprops
from skimage import measure, morphology
from skimage.filters import sobel
from skimage.segmentation import watershed
import matplotlib.pyplot  as plt

Using TensorFlow backend.


In [2]:
#Normal : Label 0 Apoptosis : Label 1 Division : Label 2 Macrocheate : Label 3 Non MatureP1 : Label 4 Mature Event: Label 5

In [3]:
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'

DenoisingModel = 'BorialisS1S2FlorisMidNoiseModel'
SegmentationMaskModel = 'DrosophilaMaskSegmentationCARE'
SegmentationStarModel = 'DrosophilaSegmentationSmartSeeds'

model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)
NoiseModel = CARE(config = None, name = DenoisingModel, basedir = ModelDirectory)






Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [16]:

Dir = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neat/StaticNonMatureP1EventTrainingDataBin2/ShiftRXY/'
cellX = 0.685185185185185
cellY = 0.314814814814815
trainlabel = 4

In [17]:
axis_norm = (0,1)
TotalCategories = 6
Label = np.zeros([TotalCategories + 4])
Label[trainlabel] = 1

Subdir = next(os.walk(Dir))
for x in Subdir[1]:
    currentdir = Dir  + x
    Path(currentdir + '/Segmentation/').mkdir(exist_ok = True)
    Raw_path = os.path.join(currentdir, '*tif')
    X = glob.glob(Raw_path)
    for fname in X:
        image = imread(fname)
        #Get Name
        Name = os.path.basename(os.path.splitext(fname)[0])



        x = image

        originalX = x.shape[0]
        originalY = x.shape[1]
       
        #Stardist Prediction

        scale_percent = 400 # percent of original size
        width = int(x.shape[1] * scale_percent / 100)
        height = int(x.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        x = cv2.resize(x, dim)
        x = normalize(x,1,99.8,axis=axis_norm)
       

        MidImage, details = model.predict_instances(x)
       
        

        segimage = MidImage
       

        #Save different method segmentation

        imwrite((currentdir + '/Segmentation/'  + Name+ '.tif' ) , segimage.astype('float32'))
     
        smallimg = segimage.astype('uint16')
        centralLabel = smallimg[int(smallimg.shape[0]*cellY), int(smallimg.shape[1]*cellX)]
        
        
        if len(regionprops(smallimg)) > 0:
         for region in regionprops(smallimg):
           
            if region.label == centralLabel and centralLabel > 0:
              minr, minc, maxr, maxc = region.bbox
              
              center = region.centroid
              height = 0.25 * abs(maxc - minc)
              width = 0.25 * abs(maxr - minr)
            if centralLabel == 0:
            
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
               
        else:
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
            
              
        print(centralLabel, Name, 0.25 * center[0]/originalY, 0.25 * center[1]/originalX, height/originalY, width/originalX)
            
        Label[TotalCategories] = 0.25 * center[1]/originalX
        Label[TotalCategories + 1] = 0.25 * center[0]/originalY
       
        Label[TotalCategories + 2] = height/originalY
        Label[TotalCategories + 3] = width/originalX
        
        writer = csv.writer(open(currentdir + '/' + (Name) + 'DetectionLabel'  +".csv", "w"))
   
        for l in Label : writer.writerow ([l])
        
        

17 39Bin2 0.2965274567216315 0.6117789078954127 0.14814814814814814 0.1388888888888889
15 51Bin2 0.321058091286307 0.700879821730444 0.09259259259259259 0.09259259259259259
16 30Bin2 0.355762012012012 0.6289758508508508 0.17592592592592593 0.25
27 36Bin2 0.34574915824915825 0.6750841750841751 0.13425925925925927 0.125
21 19Bin2 0.25189343667604536 0.7791779476562085 0.21296296296296297 0.26851851851851855
22 54Bin2 0.2952977487291213 0.6268853695324283 0.14351851851851852 0.1574074074074074
0 3Bin2 0.314814814814815 0.685185185185185 0.18518518518518517 0.18518518518518517
0 41Bin2 0.314814814814815 0.685185185185185 0.18518518518518517 0.18518518518518517
10 14Bin2 0.31752673175267315 0.6754803967147063 0.08333333333333333 0.09259259259259259
1 15Bin2 0.32676836672738313 0.6706511839708561 0.09259259259259259 0.08796296296296297
20 2Bin2 0.29048681645634555 0.65418846824664 0.10185185185185185 0.12037037037037036
19 31Bin2 0.3339773312725399 0.6770350334878928 0.1388888888888889 0.129

3 42Bin2 0.3463955026455026 0.688260582010582 0.16203703703703703 0.1111111111111111
20 12Bin2 0.3166061156524372 0.6794034043125777 0.1527777777777778 0.19907407407407407
28 25Bin2 0.3274134790528233 0.6691180935033394 0.13425925925925927 0.13425925925925927
15 22Bin2 0.3121059889676911 0.6930161544523247 0.14351851851851852 0.07407407407407407
17 48Bin2 0.28869774805041437 0.8087866247225785 0.3333333333333333 0.3055555555555556
0 24Bin2 0.314814814814815 0.685185185185185 0.18518518518518517 0.18518518518518517
6 28Bin2 0.4463930264805869 0.7423790033995348 0.18055555555555555 0.28703703703703703
20 18Bin2 0.32512188863228125 0.6937922333418869 0.11574074074074074 0.09722222222222222
27 7Bin2 0.27181069958847737 0.6717632162076607 0.14814814814814814 0.10648148148148148
0 21Bin2 0.314814814814815 0.685185185185185 0.18518518518518517 0.18518518518518517
10 29Bin2 0.3757216284026647 0.5577941766357566 0.4027777777777778 0.32407407407407407
21 34Bin2 0.3251905751905752 0.6433067683067